<a href="https://colab.research.google.com/github/whyeon92/ESAA_YB/blob/mini_project/miniproject3_MLR_Ridge_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#파일 경로 지정 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/ESAAData/miniproject3_data/"

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv(path + "movies_train.csv")
test = pd.read_csv(path + "movies_test.csv")
smpl_sub = pd.read_csv(path + "submission.csv")

In [4]:
# distributor 중복 배급사 전처리
def distr_preprocessing(x) :
    if 'CGV' in x or '무비꼴라쥬' in x :
        return 'CGV'
    elif 'E&M' in x or 'CJ엔터테인먼트' in x or 'CJ 엔터테인먼트' in x:
        return 'CJ E&M'
    elif '롯데' in x :
        return '롯데엔터테인먼트'
    elif '스폰지' in x :
        return '스폰지'
    elif '싸이더스' in x or 'KT' in x: ## 위지윅스튜디오
        return '싸이더스'
    elif '마운틴픽처스' in x or '마운틴픽쳐스' in x :
        return '마운틴픽쳐스'
    elif '메가박스' in x or '씨너스' in x:
        return '메가박스플러스엠'
    elif '쇼박스' in x or '(주)쇼박스' in x : ## 오리온 그룹
        return '쇼박스'
    elif '조이앤시네마' in x or '조이앤컨텐츠그룹' in x or '(주) 케이알씨지' in x or '드림팩트' in x:
        return '조이앤'
    elif '타임스토리' in x :
        return '타임스토리'
    elif '프리비젼' in x :
        return '프리비젼'
    elif '리틀빅픽' in x :
        return '리틀빅픽쳐스'
    elif '콘텐츠판다' in x or '(주)NEW' in x:
        return 'NEW'
    elif 'SK' in x:
        return 'SK'
    elif '마인스' in x:
        return '마인스'
    elif '에이원' in x:
        return '에이원'
    else :
        return x

In [5]:
full_data =  pd.concat([train, test])
full_data['dir_prev_bfnum'].fillna(0, inplace = True) #결측치 처리
full_data['release_time']=pd.to_datetime(full_data['release_time'])
#release_time 년,월별 데이터 만들기
full_data['release_year'] = full_data['release_time'].dt.year
full_data['release_month'] = full_data['release_time'].dt.month
#불필요한 피처 제거
full_data = full_data.drop(['title','director','release_time'], axis = 1)

full_data['distributor']=full_data['distributor'].apply(distr_preprocessing) #겹치는 배급사 해결
total_dist = full_data.distributor.value_counts()[:14]

def distributor_etc(x) :
  if x in total_dist:
    return x
  else:
    return("기타")

full_data['distributor'] = full_data['distributor'].apply(distributor_etc).apply(lambda x : distributor_etc(x))  #전체 데이터 기준으로 삼아서 배급사 칼럼 기타 레벨 처리
full_data = pd.get_dummies(full_data, columns = ['genre','distributor','screening_rat','release_year','release_month'],drop_first=True, dtype = int) #원핫 인코딩 처리
train = full_data[:600] #데이터 나눔
test = full_data[600:]
test = test.drop('box_off_num', axis = 1)  #train test 합치면서 생긴 test의 y값(NaN) 제거
X_train = train.drop('box_off_num', axis = 1) #train 피처
y_train = train['box_off_num'] #train y
y_train_log = np.log1p(train['box_off_num']) #train y log변환

In [71]:
y_train

0        23398.0
1      7072501.0
2      6959083.0
3       217866.0
4       483387.0
         ...    
595    1475091.0
596    1716438.0
597       2475.0
598    2192525.0
599    7166532.0
Name: box_off_num, Length: 600, dtype: float64

In [70]:
y_train_log

0      10.060449
1      15.771725
2      15.755558
3      12.291640
4      13.088575
         ...    
595    14.204231
596    14.355762
597     7.814400
598    14.600565
599    15.784933
Name: box_off_num, Length: 600, dtype: float64

In [6]:
print(X_train.shape)
print(test.shape)

(600, 49)
(243, 49)


In [7]:
full_data.columns

Index(['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor',
       'box_off_num', 'genre_공포', 'genre_느와르', 'genre_다큐멘터리', 'genre_드라마',
       'genre_멜로/로맨스', 'genre_뮤지컬', 'genre_미스터리', 'genre_서스펜스', 'genre_애니메이션',
       'genre_액션', 'genre_코미디', 'distributor_(주)키노아이', 'distributor_CGV',
       'distributor_CJ E&M', 'distributor_NEW', 'distributor_골든타이드픽처스',
       'distributor_기타', 'distributor_롯데엔터테인먼트', 'distributor_마운틴픽쳐스',
       'distributor_쇼박스', 'distributor_시네마달', 'distributor_싸이더스',
       'distributor_어뮤즈', 'distributor_인디스토리', 'distributor_조이앤',
       'screening_rat_15세 관람가', 'screening_rat_전체 관람가',
       'screening_rat_청소년 관람불가', 'release_year_2011', 'release_year_2012',
       'release_year_2013', 'release_year_2014', 'release_year_2015',
       'release_month_2', 'release_month_3', 'release_month_4',
       'release_month_5', 'release_month_6', 'release_month_7',
       'release_month_8', 'release_month_9', 'release_month_10',
       'release_month_11', '

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 0 to 599
Data columns (total 49 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   time                    600 non-null    int64  
 1   dir_prev_bfnum          600 non-null    float64
 2   dir_prev_num            600 non-null    int64  
 3   num_staff               600 non-null    int64  
 4   num_actor               600 non-null    int64  
 5   genre_공포                600 non-null    int64  
 6   genre_느와르               600 non-null    int64  
 7   genre_다큐멘터리             600 non-null    int64  
 8   genre_드라마               600 non-null    int64  
 9   genre_멜로/로맨스            600 non-null    int64  
 10  genre_뮤지컬               600 non-null    int64  
 11  genre_미스터리              600 non-null    int64  
 12  genre_서스펜스              600 non-null    int64  
 13  genre_애니메이션             600 non-null    int64  
 14  genre_액션                600 non-null    int64  

In [9]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

mm = MinMaxScaler()
X_train_mm = X_train.copy()
X_train_mm[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']] = mm.fit_transform(X_train_mm[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']])
X_test_mm = test.copy()
X_test_mm[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']] = mm.transform(X_test_mm[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']])

ss = StandardScaler()
X_train_ss = X_train.copy()
X_train_ss[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']] = ss.fit_transform(X_train_ss[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']])
X_test_ss = test.copy()
X_test_ss[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']] = ss.transform(X_test_ss[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']])

In [10]:
X_train

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,release_month_3,release_month_4,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12
0,96,0.00,0,91,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,130,1161602.50,2,387,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,123,220775.25,4,343,4,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,101,23894.00,2,20,6,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,108,1.00,1,251,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,111,3833.00,1,510,7,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
596,127,496061.00,1,286,6,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
597,99,0.00,0,123,4,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
598,102,0.00,0,431,4,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [11]:
X_train_mm

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,release_month_3,release_month_4,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12
0,0.377778,0.000000e+00,0.0,0.104718,0.08,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.629630,6.594276e-02,0.4,0.445339,0.12,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.577778,1.253314e-02,0.8,0.394707,0.16,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0.414815,1.356433e-03,0.4,0.023015,0.24,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.466667,5.676879e-08,0.2,0.288838,0.08,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.488889,2.175948e-04,0.2,0.586881,0.28,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
596,0.607407,2.816078e-02,0.2,0.329114,0.24,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
597,0.400000,0.000000e+00,0.0,0.141542,0.16,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
598,0.422222,0.000000e+00,0.0,0.495972,0.16,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [12]:
X_train_ss

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,release_month_3,release_month_4,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12
0,-0.268953,-0.361285,-0.741416,-0.363216,-0.698066,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1.611324,0.526530,0.950027,1.425124,-0.289043,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1.224208,-0.192546,2.641471,1.159290,0.119980,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0.007558,-0.343023,0.950027,-0.792176,0.938026,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.394674,-0.361284,0.104306,0.603454,-0.698066,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.560581,-0.358356,0.104306,2.168252,1.347050,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
596,1.445417,0.017855,0.104306,0.814913,0.938026,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
597,-0.103047,-0.361285,-0.741416,-0.169882,0.119980,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
598,0.062860,-0.361285,-0.741416,1.690959,0.119980,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
test

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,release_month_3,release_month_4,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12
0,125,3.005290e+05,2,304,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,113,3.427002e+05,4,275,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,115,4.206611e+06,3,419,7,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,116,6.913420e+05,2,408,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,110,3.173800e+04,1,380,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,78,2.590000e+03,1,4,4,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
239,86,0.000000e+00,0,18,2,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
240,107,0.000000e+00,0,224,4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
241,102,5.069900e+04,1,68,7,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [14]:
X_test_mm

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,release_month_3,release_month_4,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12
0,0.592593,0.017061,0.4,0.349827,0.12,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.503704,0.019455,0.8,0.316456,0.12,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.518519,0.238804,0.6,0.482163,0.28,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0.525926,0.039247,0.4,0.469505,0.08,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.481481,0.001802,0.2,0.437284,0.04,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.244444,0.000147,0.2,0.004603,0.16,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
239,0.303704,0.000000,0.0,0.020713,0.08,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
240,0.459259,0.000000,0.0,0.257768,0.16,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
241,0.422222,0.002878,0.2,0.078251,0.28,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [15]:
X_test_ss

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,release_month_3,release_month_4,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12
0,1.334812,-0.131590,0.950027,0.923664,-0.289043,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.671185,-0.099359,2.641471,0.748455,-0.289043,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.781790,2.853837,1.795749,1.618458,1.347050,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0.837092,0.167109,0.950027,1.552000,-0.698066,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.505278,-0.337028,0.104306,1.382832,-1.107089,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,-1.264394,-0.359306,0.104306,-0.888844,0.119980,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
239,-0.821976,-0.361285,-0.741416,-0.804260,-0.698066,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
240,0.339372,-0.361285,-0.741416,0.440329,0.119980,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
241,0.062860,-0.322536,0.104306,-0.502175,1.347050,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


# MLR

In [16]:
from sklearn.linear_model import LinearRegression, Ridge

In [23]:
#X, X
MLR = LinearRegression()
MLR.fit(X_train, y_train)
pred = MLR.predict(test)

In [24]:
smpl_sub['box_off_num'] = pred
smpl_sub.to_csv(path + 'submission_MLR_X_X.csv', index=False)

In [25]:
pred

array([ 1.46617423e+06,  2.29657023e+06,  2.93727779e+06,  2.70152727e+06,
        1.54852124e+06, -3.78060896e+05, -7.13823507e+05, -7.70716540e+05,
       -6.26881769e+05,  4.57745782e+05,  1.42193990e+05,  7.20186234e+05,
        9.31806872e+05, -6.21878097e+05,  4.97854113e+05,  1.86393812e+06,
        1.68425168e+05,  2.35145533e+06,  1.14244871e+06,  7.95598117e+05,
       -4.72430080e+05, -1.85982539e+05,  1.93773750e+05,  1.91174207e+05,
       -5.37980110e+05,  3.12411194e+06, -1.21556529e+06,  1.52129481e+06,
       -5.80548767e+05, -2.90478971e+05,  9.91467076e+05, -2.88512646e+05,
        2.94826054e+05,  1.80048402e+06,  6.08323193e+05, -9.91683172e+04,
        1.09447337e+05,  3.11289870e+05, -3.02705507e+05, -8.93165061e+05,
        2.73620172e+05, -7.98594345e+04,  1.66322181e+06, -1.40958951e+05,
       -1.28386417e+05,  2.07585337e+06,  1.83935853e+05,  6.62317086e+05,
        1.20855665e+06, -4.44237528e+04,  1.82015654e+03,  1.34358764e+06,
        3.29435572e+05,  

In [72]:
#X, log
MLR = LinearRegression()
MLR.fit(X_train, y_train_log)
pred = MLR.predict(test)
final = np.exp(pred) - 1

In [73]:
smpl_sub['box_off_num'] = final
smpl_sub.to_csv(path + 'submission_MLR_X_log.csv', index=False)

In [74]:
pred

array([12.32459701, 13.80659172, 13.7840511 , 14.22824903, 14.04244455,
        8.67557923,  7.3841029 ,  7.26724204,  6.45707444, 12.14549961,
       11.03431842, 11.49270446, 10.41907037,  7.94529401, 11.55257796,
       15.50134983,  7.90888112, 13.97802556, 10.88863672, 10.09559084,
        6.52460687,  8.15235967, 11.70964841,  7.93984453,  7.50954678,
       14.33694209,  6.88692947, 12.5988709 ,  9.34339535,  9.25320192,
       12.03582566,  7.46427193,  8.87883795, 11.20603674,  7.94155743,
        6.82712628,  8.30282067,  8.30753646,  7.63106998,  6.76143662,
        9.69021427,  8.96992035, 10.24898309, 10.11687202, 11.29748212,
       13.52636711,  8.40549827, 10.47730056, 10.48304486,  8.53038789,
        7.55046064, 13.30073358,  9.82850562, 12.82906942, 13.07431953,
       10.31298932,  9.74940506, 13.12521047, 14.6359472 , 13.9515213 ,
       13.09516812, 14.30974651,  9.53521092,  8.13559514,  9.80912049,
        7.06979968, 13.72174269,  9.78654384, 11.95269158,  9.65

In [22]:
final

array([2.25165860e+05, 9.91119815e+05, 9.69029247e+05, 1.51094882e+06,
       1.25474604e+06, 5.85709197e+03, 1.60918265e+03, 1.43159396e+03,
       6.36194174e+02, 1.88244005e+05, 6.19635932e+04, 9.79972066e+04,
       3.34912841e+04, 2.82126210e+03, 1.04043915e+05, 5.39697757e+06,
       2.72034391e+03, 1.17646496e+06, 5.35632273e+04, 2.42349135e+04,
       6.80711717e+02, 3.47056115e+03, 1.21739671e+05, 2.80592407e+03,
       1.82438605e+03, 1.68443625e+06, 9.78389551e+02, 2.96222910e+05,
       1.14221280e+04, 1.04369336e+04, 1.68690294e+05, 1.74358491e+03,
       7.17744421e+03, 7.35722465e+04, 2.81073620e+03, 9.21535891e+02,
       4.03423843e+03, 4.05331273e+03, 2.06025433e+03, 8.62882376e+02,
       1.61577062e+04, 7.86197526e+03, 2.82527957e+04, 2.47562095e+04,
       8.06173941e+04, 7.48903772e+05, 4.47058527e+03, 3.54994468e+04,
       3.57039587e+04, 5.06541068e+03, 1.90061850e+03, 5.97632866e+05,
       1.85542050e+04, 3.72900435e+05, 4.76544998e+05, 3.01203445e+04,
      

In [28]:
#minmax, X
MLR = LinearRegression()
MLR.fit(X_train_mm, y_train)
pred = MLR.predict(X_test_mm)

In [29]:
smpl_sub['box_off_num'] = pred
smpl_sub.to_csv(path + 'submission_MLR_MM_X.csv', index=False)

In [30]:
#minmax, log
MLR = LinearRegression()
MLR.fit(X_train_mm, y_train_log)
pred = MLR.predict(X_test_mm)
final = np.exp(pred) - 1

In [31]:
smpl_sub['box_off_num'] = final
smpl_sub.to_csv(path + 'submission_MLR_MM_log.csv', index=False)

In [32]:
#Standard, X
MLR = LinearRegression()
MLR.fit(X_train_ss, y_train)
pred = MLR.predict(X_test_ss)

In [33]:
smpl_sub['box_off_num'] = pred
smpl_sub.to_csv(path + 'submission_MLR_SS_X.csv', index=False)

In [34]:
#Standard, log
MLR = LinearRegression()
MLR.fit(X_train_ss, y_train_log)
pred = MLR.predict(X_test_ss)
final = np.exp(pred) - 1

In [35]:
smpl_sub['box_off_num'] = final
smpl_sub.to_csv(path + 'submission_MLR_SS_log.csv', index=False)

# Ridge

In [36]:
#X, X
ridge = Ridge()
ridge.fit(X_train, y_train)
pred = ridge.predict(test)

In [37]:
smpl_sub['box_off_num'] = pred
smpl_sub.to_csv(path + 'submission_Ridge_X_X.csv', index=False)

In [38]:
#X, log
ridge = Ridge()
ridge.fit(X_train, y_train_log)
pred = ridge.predict(test)
final = np.exp(pred) - 1

In [39]:
smpl_sub['box_off_num'] = final
smpl_sub.to_csv(path + 'submission_Ridge_X_log.csv', index=False)

In [40]:
#minmax, X
ridge = Ridge()
ridge.fit(X_train_mm, y_train)
pred = ridge.predict(X_test_mm)

In [41]:
smpl_sub['box_off_num'] = pred
smpl_sub.to_csv(path + 'submission_Ridge_MM_X.csv', index=False)

In [42]:
#minmax, log
ridge = Ridge()
ridge.fit(X_train_mm, y_train_log)
pred = ridge.predict(X_test_mm)
final = np.exp(pred) - 1

In [43]:
smpl_sub['box_off_num'] = final
smpl_sub.to_csv(path + 'submission_Ridge_MM_log.csv', index=False)

In [44]:
#Standard, X
ridge = Ridge()
ridge.fit(X_train_ss, y_train)
pred = ridge.predict(X_test_ss)

In [45]:
smpl_sub['box_off_num'] = pred
smpl_sub.to_csv(path + 'submission_Ridge_SS_X.csv', index=False)

In [46]:
#Standard, log
ridge = Ridge()
ridge.fit(X_train_ss, y_train_log)
pred = ridge.predict(X_test_ss)
final = np.exp(pred) - 1

In [47]:
smpl_sub['box_off_num'] = final
smpl_sub.to_csv(path + 'submission_Ridge_SS_log.csv', index=False)

## Ridge GridSearch

In [54]:
from sklearn.model_selection import GridSearchCV

In [55]:
ridge_alpha = {'alpha': [0.01, 0.05, 0.1, 0.2, 0.5, 1, 3, 10]}

In [56]:
#X, X
ridge = Ridge()
grid_ridge = GridSearchCV(ridge, param_grid = ridge_alpha, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1, refit=True)
grid_ridge.fit(X_train, y_train)
pred = grid_ridge.predict(test)

print('최적 하이퍼 파라미터:\n', grid_ridge.best_params_)
print('최고 예측 RMSE: {0:.4f}'.format(-1* grid_ridge.best_score_))

최적 하이퍼 파라미터:
 {'alpha': 10}
최고 예측 RMSE: 1438381.4194


In [57]:
smpl_sub['box_off_num'] = pred
smpl_sub.to_csv(path + 'submission_Grid_X_X.csv', index=False)

In [58]:
#X, log
ridge = Ridge()
grid_ridge = GridSearchCV(ridge, param_grid = ridge_alpha, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1, refit=True)
grid_ridge.fit(X_train, y_train_log)
pred = grid_ridge.predict(test)

print('최적 하이퍼 파라미터:\n', grid_ridge.best_params_)
print('최고 예측 RMSE: {0:.4f}'.format(-1* grid_ridge.best_score_))

final = np.exp(pred) - 1

최적 하이퍼 파라미터:
 {'alpha': 3}
최고 예측 RMSE: 1.9191


In [59]:
smpl_sub['box_off_num'] = final
smpl_sub.to_csv(path + 'submission_Grid_X_log.csv', index=False)

In [60]:
#minmax, X
ridge = Ridge()
grid_ridge = GridSearchCV(ridge, param_grid = ridge_alpha, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1, refit=True)
grid_ridge.fit(X_train_mm, y_train)
pred = grid_ridge.predict(X_test_mm)

print('최적 하이퍼 파라미터:\n', grid_ridge.best_params_)
print('최고 예측 RMSE: {0:.4f}'.format(-1* grid_ridge.best_score_))

최적 하이퍼 파라미터:
 {'alpha': 3}
최고 예측 RMSE: 1449648.2549


In [61]:
smpl_sub['box_off_num'] = pred
smpl_sub.to_csv(path + 'submission_Grid_MM_X.csv', index=False)

In [62]:
#minmax, log
ridge = Ridge()
grid_ridge = GridSearchCV(ridge, param_grid = ridge_alpha, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1, refit=True)
grid_ridge.fit(X_train_mm, y_train_log)
pred = grid_ridge.predict(X_test_mm)

print('최적 하이퍼 파라미터:\n', grid_ridge.best_params_)
print('최고 예측 RMSE: {0:.4f}'.format(-1* grid_ridge.best_score_))

final = np.exp(pred) - 1

최적 하이퍼 파라미터:
 {'alpha': 1}
최고 예측 RMSE: 1.9168


In [63]:
smpl_sub['box_off_num'] = final
smpl_sub.to_csv(path + 'submission_Grid_MM_log.csv', index=False)

In [64]:
#Standard, X
ridge = Ridge()
grid_ridge = GridSearchCV(ridge, param_grid = ridge_alpha, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1, refit=True)
grid_ridge.fit(X_train_ss, y_train)
pred = grid_ridge.predict(X_test_ss)

print('최적 하이퍼 파라미터:\n', grid_ridge.best_params_)
print('최고 예측 RMSE: {0:.4f}'.format(-1* grid_ridge.best_score_))

최적 하이퍼 파라미터:
 {'alpha': 10}
최고 예측 RMSE: 1437354.9680


In [65]:
smpl_sub['box_off_num'] = pred
smpl_sub.to_csv(path + 'submission_Grid_SS_X.csv', index=False)

In [66]:
#Standard, log
ridge = Ridge()
grid_ridge = GridSearchCV(ridge, param_grid = ridge_alpha, scoring = 'neg_root_mean_squared_error', cv = 5, n_jobs = -1, refit=True)
grid_ridge.fit(X_train_ss, y_train_log)
pred = grid_ridge.predict(X_test_ss)

print('최적 하이퍼 파라미터:\n', grid_ridge.best_params_)
print('최고 예측 RMSE: {0:.4f}'.format(-1* grid_ridge.best_score_))

final = np.exp(pred) - 1

최적 하이퍼 파라미터:
 {'alpha': 3}
최고 예측 RMSE: 1.9183


In [67]:
smpl_sub['box_off_num'] = final
smpl_sub.to_csv(path + 'submission_Grid_SS_log.csv', index=False)